In [1]:
pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras import layers, optimizers
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score, mean_squared_error, mean_absolute_percentage_error
import keras_tuner as kt
from tensorflow.keras.metrics import MeanAbsoluteError, RootMeanSquaredError
import shap
import joblib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Research Methodology/Dataset and Training File/All_Cities_AirQuality_Climate_Data - LSTM.csv")
df

,valid_time,city,bcaod550,omaod550,pm1,pm2p5,pm10,aod550,tcco,tcno2,gtco3,tcso2,u10,v10,d2m,t2m,sp,blh,tp,ssrd
0,2020-01-01 00:00:00,Delhi,0.035246,0.689838,3.274958e-07,3.808820e-07,5.396277e-07,1.072968,0.001506,0.000009,0.007481,0.000010,-2.029889,-0.134339,277.574695,282.634145,99427.71400,242.457083,0.000000e+00,1.070848e+06
1,2020-01-01 03:00:00,Delhi,0.039080,0.636663,4.424187e-07,5.134427e-07,7.259009e-07,0.949349,0.001634,0.000008,0.007827,0.000013,-2.341974,-0.749627,280.646973,283.305780,99416.79500,53.517427,0.000000e+00,5.913830e+04
2,2020-01-01 06:00:00,Delhi,0.029610,0.534700,3.504060e-07,4.071628e-07,5.770834e-07,0.768096,0.001683,0.000010,0.007807,0.000012,-1.907452,-0.808675,281.741357,286.553617,99413.22975,459.425050,1.225489e-05,1.069370e+06
3,2020-01-01 09:00:00,Delhi,0.035891,0.629083,2.922284e-07,3.391009e-07,4.792977e-07,0.945668,0.001658,0.000012,0.007543,0.000016,-1.730567,-0.939833,283.797203,287.204932,99256.58250,70.177471,3.792069e-06,4.536938e+04
4,2020-01-01 12:00:00,Delhi,0.021761,0.431958,2.229857e-07,2.594867e-07,3.679207e-07,0.634090,0.001368,0.000008,0.007003,0.000014,-0.688096,-0.624323,283.080620,287.859692,99185.51500,407.002452,3.147673e-06,9.787946e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10771,2021-03-24 09:00:00,Jakarta,0.010291,0.173769,4.087564e-08,4.853614e-08,6.917716e-08,0.331794,0.000829,0.000005,0.005574,0.000004,0.440630,1.458182,295.804017,298.011008,98719.67125,84.702618,1.887121e-04,0.000000e+00
10772,2021-03-24 12:00:00,Jakarta,0.009570,0.141390,2.704103e-08,3.333481e-08,4.815237e-08,0.287791,0.000826,0.000003,0.005580,0.000003,1.728188,1.923312,296.325315,301.629450,98610.37625,533.720505,5.956036e-04,7.866938e+05
10773,2021-03-24 15:00:00,Jakarta,0.007929,0.119112,4.187835e-08,4.980510e-08,7.108394e-08,0.245235,0.000772,0.000004,0.005533,0.000003,1.275100,1.421648,295.107145,298.608860,98644.53250,148.936168,1.217628e-04,0.000000e+00
10774,2021-03-24 18:00:00,Jakarta,0.006401,0.101479,2.634174e-08,3.191130e-08,4.584265e-08,0.222353,0.000720,0.000002,0.005665,0.000003,1.778162,1.467679,296.496280,302.926283,98540.50125,730.951925,2.532402e-05,1.701658e+06


In [5]:
df['valid_time'] = pd.to_datetime(df['valid_time'])
df = df.sort_values(by=['valid_time', 'city']).reset_index(drop=True)

In [6]:
missing_value = 100 * (df.isna().sum() / len(df))
print(missing_value)

valid_time    0.0
city          0.0
bcaod550      0.0
omaod550      0.0
pm1           0.0
pm2p5         0.0
pm10          0.0
aod550        0.0
tcco          0.0
tcno2         0.0
gtco3         0.0
tcso2         0.0
u10           0.0
v10           0.0
d2m           0.0
t2m           0.0
sp            0.0
blh           0.0
tp            0.0
ssrd          0.0
dtype: float64


In [7]:
df['hour'] = df['valid_time'].dt.hour
df['days'] = df['valid_time'].dt.dayofweek
df['month'] = df['valid_time'].dt.month

In [8]:
df = pd.get_dummies(df, columns=['city'])

In [9]:
df['pm2p5'] *= 1e07
df = df.drop(columns=['pm1', 'pm10'], axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10776 entries, 0 to 10775
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   valid_time     10776 non-null  datetime64[ns]
 1   bcaod550       10776 non-null  float64       
 2   omaod550       10776 non-null  float64       
 3   pm2p5          10776 non-null  float64       
 4   aod550         10776 non-null  float64       
 5   tcco           10776 non-null  float64       
 6   tcno2          10776 non-null  float64       
 7   gtco3          10776 non-null  float64       
 8   tcso2          10776 non-null  float64       
 9   u10            10776 non-null  float64       
 10  v10            10776 non-null  float64       
 11  d2m            10776 non-null  float64       
 12  t2m            10776 non-null  float64       
 13  sp             10776 non-null  float64       
 14  blh            10776 non-null  float64       
 15  tp             1077

In [11]:
df = df.drop(columns=['valid_time'])

In [12]:
target_col = 'pm2p5'
features_col = [col for col in df.columns if col != target_col]

In [13]:
data_split = int(len(df) * 0.8)
df_train = df.iloc[:data_split]
df_test = df.iloc[data_split:]

In [14]:
minmax_X = MinMaxScaler()
minmax_y = MinMaxScaler()

X_train = minmax_X.fit_transform(df_train[features_col])
X_test = minmax_X.transform(df_test[features_col])
y_train = minmax_y.fit_transform(df_train[[target_col]])
y_test = minmax_y.transform(df_test[[target_col]])

In [15]:
joblib.dump(minmax_X, 'LSTM_minmax_x.pkl')
joblib.dump(minmax_y, 'LSTM_minmax_y.pkl')

['LSTM_minmax_y.pkl']

In [16]:
def sequence(feature, target, window_size=12):
  X, y = [], []
  for i in range(len(feature) - window_size):
    X.append(feature[i:i+window_size])
    y.append(target[i+window_size])
  return np.array(X), np.array(y)

window = 12
X_train_sq, y_train_sq = sequence(X_train, y_train, window)
X_test_sq, y_test_sq = sequence(X_test, y_test, window)

In [17]:
print(X_train_sq.shape)
print(y_train_sq.shape)
print(X_test_sq.shape)
print(y_test_sq.shape)

(8608, 12, 21)
(8608, 1)
(2144, 12, 21)
(2144, 1)


In [18]:
model = keras.models.Sequential()

In [19]:
# First Layer
# 64 = number of memory cells
# return_sequence = giving the
# input_shape = to give the models the expected input data
model.add(layers.LSTM(64, return_sequences=True, input_shape=(X_train_sq.shape[1], X_train_sq.shape[2])))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
model.add(layers.LSTM(64, return_sequences=False))

In [21]:
# 128 = neural network layer with 128 neuron
model.add(layers.Dense(128, activation='relu'))

In [22]:
# this layer randomly drops 50% of the neuron to prevent overfitting
model.add(layers.Dropout(0.5))

In [23]:
model.add(layers.Dense(1))

In [24]:
model.compile(optimizer='adam', loss='mae', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

In [25]:
model.fit(X_train_sq, y_train_sq, epochs=20, batch_size=32)

Epoch 1/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.1111 - mean_absolute_error: 0.1111 - root_mean_squared_error: 0.1755
Epoch 2/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0553 - mean_absolute_error: 0.0553 - root_mean_squared_error: 0.0975
Epoch 3/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0520 - mean_absolute_error: 0.0520 - root_mean_squared_error: 0.0912
Epoch 4/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0509 - mean_absolute_error: 0.0509 - root_mean_squared_error: 0.0902
Epoch 5/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0495 - mean_absolute_error: 0.0495 - root_mean_squared_error: 0.0881
Epoch 6/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0466 - mean_absolute_error: 0.0466 - root_mean_squared_error: 0.0808
Epoch 7/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0473 - mean_absolute_error: 0.0473 - root_mean_squared_error: 0.0861
Epoch 8/20
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0490 - mean_absolute_

In [26]:
y_pred_sq = model.predict(X_test_sq)

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [27]:
y_pred_inversed = minmax_y.inverse_transform(y_pred_sq)
y_pred_real = minmax_y.inverse_transform(y_test_sq)

In [28]:
print(f"RMSE Score: {root_mean_squared_error(y_pred_real, y_pred_inversed)}")
print(f"MAE Score: {mean_absolute_error(y_pred_real, y_pred_inversed)}")
print(f"R2 Score: {r2_score(y_pred_real, y_pred_inversed)}")

RMSE Score: 0.40815104991659185
MAE Score: 0.2265472905055367
R2 Score: 0.8506547041111934


In [29]:
model.save("LSTM_base_model.keras")

In [30]:
def LSTM_model_builder(hp):
  model = keras.Sequential()
  model.add(
      layers.LSTM(
          units=hp.Int('unit_1', min_value=32, max_value=256, step=32),
          return_sequences=True,
          input_shape=(X_train_sq.shape[1], X_train_sq.shape[2]),
      )
  )

  for i in range(hp.Int('n_layers', 1, 4)):
    model.add(
        layers.LSTM(
            hp.Int(f"{i} units", min_value=32, max_value=256, step=32),
            return_sequences=True
        )
    )

  model.add(
      layers.LSTM(
          units=hp.Int('unit_2', min_value=32, max_value=256, step=32), return_sequences=False
      )
  )

  model.add(
      layers.Dropout(
            rate = hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
      )
  )
  model.add(
      layers.Dense(
          units=hp.Int('dense', min_value=16, max_value=128, step=16),
          activation=hp.Choice('dense_act2', values=['relu', 'linear', 'sigmoid'])
      )
  )

  model.add(
      layers.Dense(
          1,
          activation=hp.Choice('dense_act2', values=['relu', 'linear', 'sigmoid'])
      )
  )

  model.compile(
      optimizer=optimizers.Adam(learning_rate=hp.Choice('lr', [1e-3, 5e-4, 1e-4])),
      loss='mae',
      metrics=[RootMeanSquaredError(), MeanAbsoluteError()]
  )

  return model

In [31]:
tunes = kt.RandomSearch(
    LSTM_model_builder,
    objective='mean_absolute_error',
    max_trials=25,
    executions_per_trial=1,
    directory='tuning_dir_retry',
    project_name='LSTM_tuning'
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [32]:
tunes.search(
    x=X_train_sq,
    y=y_train_sq,
    epochs=50,
    batch_size=64,
)

Trial 25 Complete [00h 01m 42s]
mean_absolute_error: 0.036783937364816666

Best mean_absolute_error So Far: 0.034888435155153275
Total elapsed time: 00h 44m 50s


In [33]:
best_parameter = tunes.get_best_hyperparameters(num_trials=1)[0]

for par in best_parameter.values:
  print(f"{par}: {best_parameter.get(par)}")

unit_1: 160
n_layers: 1
0 units: 32
unit_2: 192
dropout: 0.2
dense: 128
dense_act2: relu
lr: 0.0005
1 units: 192
2 units: 32
3 units: 96


In [34]:
best_mod = tunes.hypermodel.build(best_parameter)

In [35]:
best_mod.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 12, 160)        │       116,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 12, 32)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 192)            │       172,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 338,817 (1.29 MB)

 Trainable params: 338,817 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
best_mod.fit(X_train_sq, y_train_sq, epochs=50)

Epoch 1/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.1042 - mean_absolute_error: 0.1042 - root_mean_squared_error: 0.1699
Epoch 2/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0534 - mean_absolute_error: 0.0534 - root_mean_squared_error: 0.0918
Epoch 3/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0501 - mean_absolute_error: 0.0501 - root_mean_squared_error: 0.0843
Epoch 4/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0484 - mean_absolute_error: 0.0484 - root_mean_squared_error: 0.0835
Epoch 5/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0458 - mean_absolute_error: 0.0458 - root_mean_squared_error: 0.0784
Epoch 6/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0460 - mean_absolute_error: 0.0460 - root_mean_squared_error: 0.0786
Epoch 7/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0447 - mean_absolute_error: 0.0447 - root_mean_squared_error: 0.0795
Epoch 8/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0436 - mean_absolute

In [37]:
y_pred_best = best_mod.predict(X_test_sq)
y_pred_best_train = best_mod.predict(X_train_sq)

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [38]:
y_train_inverse = minmax_y.inverse_transform(y_train_sq)
y_test_inverse = minmax_y.inverse_transform(y_test_sq)

y_train_pred_inv = minmax_y.inverse_transform(y_pred_best_train.reshape(-1, 1))
y_test_pred_inv = minmax_y.inverse_transform(y_pred_best.reshape(-1, 1))

In [39]:
print("LSTM Best Model Test RMSE Score: ", root_mean_squared_error(y_test_sq, y_pred_best))
print("LSTM Best Model Test MAE Score: ", mean_absolute_error(y_test_sq, y_pred_best))
print("LSTM Best Model Test R2 Score: ", r2_score(y_test_sq, y_pred_best))
print("LSTM Best Model Test MAPE Score: ", mean_absolute_percentage_error(y_test_pred_inv, y_test_inverse))

LSTM Best Model Test RMSE Score:  0.06797271644337693
LSTM Best Model Test MAE Score:  0.037705869618192264
LSTM Best Model Test R2 Score:  0.8464077874096341
LSTM Best Model Test MAPE Score:  0.33722312082327827


In [40]:
print("LSTM Best Model Train RMSE Score: ", root_mean_squared_error(y_train_sq, y_pred_best_train))
print("LSTM Best Model Train MAE Score: ", mean_absolute_error(y_train_sq, y_pred_best_train))
print("LSTM Best Model Train R2 Score: ", r2_score(y_train_sq, y_pred_best_train))
print("LSTM Best Model Train MAPE Score: ", mean_absolute_percentage_error(y_train_pred_inv, y_train_inverse))

LSTM Best Model Train RMSE Score:  0.05799547243958374
LSTM Best Model Train MAE Score:  0.032270966243674655
LSTM Best Model Train R2 Score:  0.8863828206490769
LSTM Best Model Train MAPE Score:  0.28585599507500037


In [41]:
best_mod.save('LSTM_best_model.keras')

In [42]:
from tensorflow.keras.models import load_model
best_model = load_model("/content/LSTM_best_model.keras")

In [43]:
def permutation_importance(model, X, y, metric=mean_squared_error):
    baseline = metric(y, model.predict(X))
    importances = []
    for i in range(X.shape[2]):
        X_permuted = X.copy()
        np.random.shuffle(X_permuted[:, :, i])
        score = metric(y, model.predict(X_permuted))
        importances.append(score - baseline)
    return np.array(importances)

In [44]:
importance = permutation_importance(best_model, X_test_sq, y_test_sq)

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [45]:
feature = ['bcaod550', 'omaod550', 'aod550', 'tcco',
       'tcno2', 'gtco3', 'tcso2', 'u10', 'v10', 'd2m', 't2m', 'sp', 'blh',
       'tp', 'ssrd', 'hour', 'days', 'month', 'city_Delhi', 'city_Helsinki',
       'city_Jakarta']

sorted_idx = np.argsort(importance)[::-1]
sorted_importances = importance[sorted_idx]
sorted_features = np.array(feature)[sorted_idx]

greater_than_zero_imp = sorted_importances[sorted_importances > 0]
greater_than_zero_feat = sorted_features[sorted_importances > 0]

top_n = 10
top_importances = greater_than_zero_imp[:top_n]
top_features = greater_than_zero_feat[:top_n]

plt.figure(figsize=(10, 6))
plt.barh(range(len(top_importances)), top_importances)
plt.yticks(range(len(top_features)), top_features, rotation=45)
plt.tight_layout()
plt.gca().invert_yaxis()
plt.savefig("lstm_feature_importance.png", dpi=300, bbox_inches='tight')
plt.close()

In [46]:
from google.colab import files
files.download("lstm_feature_importance.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>